In [64]:
#The following special forms using leading or trailing underscores are recognized (these can generally be combined with any case convention):

#    _single_leading_underscore: weak "internal use" indicator. E.g. from M import * does not import objects whose name starts with an underscore.

#    single_trailing_underscore_: used by convention to avoid conflicts with Python keyword, e.g.    Tkinter.Toplevel(master, class_='ClassName')

#    __double_leading_underscore: when naming a class attribute, invokes name mangling (inside class FooBar, __boo becomes _FooBar__boo; see below).

#    __double_leading_and_trailing_underscore__: "magic" objects or attributes that live in user-controlled namespaces. E.g. __init__, __import__ or __file__."""""

In [65]:
#[1,2]+[1]=[1,2,1]    <=> [1,2].__add__([1])
#len([1,2])  <=>  [1,2].__len__()
#[1,2].__add__([1]),([1,2]+[1]).__len__()


In [213]:
class mynumber:
    def __init__(self,data,_parents=(),_op=''):
        self.data=data
        self.grad=0
        
        self._backward= lambda : None
        self._prev=set(_parents)
        self._op=_op#storing what operation creates this value, so we can backtrack
    
    def __add__(self,other):
        other=other if isinstance(other,mynumber) else mynumber(other)
        out=mynumber(self.data+other.data,(self,other),'+')
        
        def _backward():
            self.grad += out.grad #out= a+b ==>gradient flows to both a,b with unit weight
            other.grad += out.grad
        out._backward=_backward
        
        return out
    
    def __mul__(self,other):
        other = other if isinstance(other,mynumber) else mynumber(other)
        out=mynumber(self.data*other.data,(self,other),'*')
        
        def _backward():
            self.grad +=out.grad * other.data
            other.grad += out.grad*self.data
        out._backward=_backward
        
        return out
    
    def __pow__(self,other):
        assert isinstance(other,(int,float))
        out=mynumber(self.data**other,(self,),f'^{other}')
        
        def _backward():
            self.grad += out.grad* other * (self.data**(other-1))
        out._backward=_backward

        return out
        
    
    def __neg__(self):
        #return self*-1 
        return self.__mul__(-1)
    def __sub__(self,other):
        #return self.__add__(other.__neg__())
        return self+(-other)
    def __radd__(self,other):
        return self+other
    def __rsub__(self,other):
        return -self + other
    def __rmul__(self,other):
        return self*other
    def __truediv__(self,other):
        return self*(other**-1)
    def __rdiv__(self,other):
        return other*(self**-1)
    
    def __repr__(self):
        return f"MyNumber(data={self.data}, grad={self.grad})"
    
    
    def relu(self):
        out=mynumber(0 if self.data<0 else self.data,(self,),'ReLU')
        def _backward():
            self.grad += (out.data >0)*out.grad
        out._backward=_backward
        
        return out

    
    def backward(self):
        topo=[]
        print("RAAGASG")
        visited=set()
        def buildtopo(v):
            if v not in  visited:
                visited.add(v)
                for parent in v._prev:
                    buildtopo(parent)
                topo.append(v)
        buildtopo(self)
        print("ASFASFASFASF")
        self.grad=1
        for v in reversed(topo):
            v._backward()

In [214]:
x=mynumber(19)
#(x*5).data, next((p.data for p in list((x*5)._prev)))

In [215]:
y=5*x
y.backward()

RAAGASG
ASFASFASFASF


In [216]:
x,y

(MyNumber(data=19, grad=5), MyNumber(data=95, grad=1))

In [217]:
import random
import numpy as np

In [218]:
class Module:
    def parameters(self):
        return []
    def zero_grad(self):
        for p in self.parameters():
            p.grad=0

class neuron(Module):
    def __init__(self,nin,nonlin=True):
        self.w=[mynumber(random.uniform(-1,1)) for _ in range(nin)]#initilise random weights for each input for this single neuron
        self.b=mynumber(0)#initialise bias
        self.nonlin=nonlin
    
    def __call__(self,x):#n1=neuron(10); neuron takes input from 10 nodes; n1(x) <==>n1.__call__(x)
        print(len(self.w))
        
        act=sum((wi*xi for wi,xi in zip(self.w,x)),self.b)#act= w*x +b
        return act.relu() if self.nonlin else act
    
    def parameters(self):
        #print('a')
        return self.w +[self.b]
    
    def __repr__(self):
        return f"{'ReLU' if self.nonlin else 'Linear'}Neuron({len(self.w)})"   

class layer(Module):
    def __init__(self, nin,nout,**kwargs):
        self.neurons=[neuron(nin,**kwargs) for _ in range(nout)]#nout number of neurons in layer
    
    def __call__(self,x):#layer(x)==> output of neurons
        out= [n(x) for n in self.neurons]
        return out[0] if len(out)==1 else out
    
    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]
    def __repr__(self):
        return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"

class Multilayernetwork(Module):
    def __init__(self,nin,nouts):
        sz=[nin]+nouts
        self.layers=[layer(sz[i],sz[i+1],nonlin=i!=len(nouts)-1) for i in range(len(nouts))]
    
    def __call__(self,x):
        for layers in self.layers:
            x=layers(x)
        return x
    
    def parameters(self):
        return [p for layers in self.layers for p in layers.parameters()]
    
    def __repr__(self):
        return f"MLP of [{', '.join(str(layers) for layers in self.layers)}]"

In [221]:
n1=neuron(2)
x=[1,2]
o1=n1(x)


2


In [228]:
o1.backward()

RAAGASG
ASFASFASFASF


In [220]:
mlp=Multilayernetwork(3,[2,2])

In [192]:
#mlp.parameters()

In [193]:
#for k1,k2 in zip(mlp.parameters(),[(mlp.parameters()[i]._prev) for i in range(len(mlp.parameters()))]):
#    print(k1)
#    print(k2)
#    print(">>>>>>>")

In [231]:
m1[0].backward()

In [225]:
mlp.zero_grad()
print([m.grad for m in mlp.parameters()])
#print(len(mlp.parameters()))
print("BEFORE")
#m1=mlp([0,0,0])
mlp.zero_grad()
m1[0].backward()#,m1[1].backward()
print([m.grad for m in mlp.parameters()]+m1)
#print(len(mlp.parameters()))


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
BEFORE
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, MyNumber(data=0.0, grad=1), MyNumber(data=0.0, grad=1)]


In [212]:
m1[0].backward()

In [121]:
#mlp.parameters(),"asd",list(list(m1[0]._prev)[0]._prev)[1]#m1._prev,list(list(m1._prev)[0]._prev)[1]._prev

In [50]:
#|um((wi*xi for wi,xi in zip(w,x)),3)
